# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv('cities.csv')
city_data.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,0,Tasiilaq,GL,65.61,-37.64,8.60,55,0,3.67
1,1,Wukari,NG,7.85,9.78,77.85,50,68,4.61
2,2,Lüderitz,NaN,-26.65,15.16,59.72,88,80,11.03
3,3,Avarua,CK,-21.21,-159.78,84.20,70,0,14.99
4,4,Bambous Virieux,MU,-20.34,57.76,75.58,73,75,11.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = city_data["Humidity"]
#maxhumidity = humidity.max()
locations = city_data[["Latitude", "Longitude"]]

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=100,point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_city_df = city_data.loc[(city_data['Temperature'] > 70) & (city_data['Temperature'] < 80)
            & (city_data['Cloudiness'] == 0) & (city_data['Wind Speed'] < 10)]
#new_city_df
city_drop_df = new_city_df.dropna()
#city_drop_df
city_drop_df.reset_index(inplace = True)
del city_drop_df['index']
city_drop_df
city_drop_df.head()

,Unnamed: 0,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,57,Belmonte,BR,-15.86,-38.88,77.00,78,0,8.05
1,81,Saint-Philippe,RE,-21.36,55.77,70.07,78,0,8.05
2,140,Sur,OM,22.57,59.53,70.11,72,0,5.44
3,146,Santa Fe,AR,-31.63,-60.70,79.00,60,0,1.01
4,239,Nelson Bay,AU,-32.72,152.15,75.27,53,0,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = city_drop_df.loc[:,['City', 'Country', 'Latitude', 'Longitude','Temperature','Cloudiness', 'Wind Speed' ]]
#hotel_df.head()
hotel_df['Hotel Name'] = ''
hotel_df.head()

,City,Country,Latitude,Longitude,Temperature,Cloudiness,Wind Speed,Hotel Name
0,Belmonte,BR,-15.86,-38.88,77.00,0,8.05,
1,Saint-Philippe,RE,-21.36,55.77,70.07,0,8.05,
2,Sur,OM,22.57,59.53,70.11,0,5.44,
3,Santa Fe,AR,-31.63,-60.70,79.00,0,1.01,
4,Nelson Bay,AU,-32.72,152.15,75.27,0,5.82,


In [7]:
#search googleplaces for hotels within 5000 meters
#list for hotel names
hotels = []
x = 0

# search through city
for cities in hotel_df['City']:
    target_coordinates = str(hotel_df['Latitude'][x])+' , '+str(hotel_df['Longitude'][x])
    x+=1
    target_radius = 5000
    target_type = 'lodging'
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# try/except    
    try:
        response = requests.get(base_url, params=params)
        hotel_results = response.json()
        hotels.append(hotel_results['results'][0]['name'])
        
    except IndexError:
        hotels.append(np.nan)

#dataframe with hotel names
hotel_df['Hotel Name'] = hotels
hotel_df
#new_hotel_df = hotel_df.dropna()

,City,Country,Latitude,Longitude,Temperature,Cloudiness,Wind Speed,Hotel Name
0,Belmonte,BR,-15.86,-38.88,77.00,0,8.05,NaN
1,Saint-Philippe,RE,-21.36,55.77,70.07,0,8.05,NaN
2,Sur,OM,22.57,59.53,70.11,0,5.44,NaN
3,Santa Fe,AR,-31.63,-60.70,79.00,0,1.01,NaN
4,Nelson Bay,AU,-32.72,152.15,75.27,0,5.82,NaN
5,Saint-Joseph,RE,-21.37,55.62,73.40,0,8.05,NaN
6,Tahoua,NE,14.89,5.27,75.20,0,2.24,NaN
7,Edd,ER,13.93,41.69,76.87,0,4.59,NaN
8,Tambura,SS,5.60,27.47,74.14,0,0.72,NaN
9,Makkah al Mukarramah,SA,21.43,39.83,71.80,0,0.94,NaN


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure


In [9]:
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))